# LLM Inference

In [8]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel
import torch

In [9]:
DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

print(DEVICE)

mps


In [13]:
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(DEVICE)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14", use_fast=False)

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

In [14]:
processor.tokenizer.model_max_length

77

In [33]:
import os

os.listdir('.')

['cats.jpg', '.ipynb_checkpoints', 'sandbox.ipynb']

In [34]:
# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open('./cats.jpg')
# image

In [52]:
model

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [37]:
model.text_model

CLIPTextTransformer(
  (embeddings): CLIPTextEmbeddings(
    (token_embedding): Embedding(49408, 512)
    (position_embedding): Embedding(77, 512)
  )
  (encoder): CLIPEncoder(
    (layers): ModuleList(
      (0-11): 12 x CLIPEncoderLayer(
        (self_attn): CLIPSdpaAttention(
          (k_proj): Linear(in_features=512, out_features=512, bias=True)
          (v_proj): Linear(in_features=512, out_features=512, bias=True)
          (q_proj): Linear(in_features=512, out_features=512, bias=True)
          (out_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): CLIPMLP(
          (activation_fn): QuickGELUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (layer_norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (final_layer_norm

In [39]:
model.text_projection

Linear(in_features=512, out_features=512, bias=False)

In [74]:
with torch.no_grad():
    inputs = processor(text=["a photo of a cat", "a photo of a dog"], return_tensors="pt", padding=True).to(DEVICE)
    outputs_text = model.text_projection(model.text_model(**inputs).pooler_output)

    inputs = processor(images=image, return_tensors="pt", padding=True).to(DEVICE)
    outputs_image = model.visual_projection(model.vision_model(**inputs).pooler_output)

    inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True).to(DEVICE)
    outputs = model(**inputs)

In [76]:
outputs_text

tensor([[ 0.1555,  0.0733, -0.2448,  ..., -0.5327, -0.4588,  0.0346],
        [ 0.0932,  0.2764, -0.4137,  ..., -0.5852, -0.2590,  0.1193]],
       device='mps:0')

In [77]:
outputs.text_embeds

tensor([[ 0.0148,  0.0070, -0.0234,  ..., -0.0508, -0.0438,  0.0033],
        [ 0.0087,  0.0258, -0.0387,  ..., -0.0547, -0.0242,  0.0112]],
       device='mps:0')

In [75]:
torch.matmul(outputs_text, outputs_image.T).softmax(dim=0) 

tensor([[0.9959],
        [0.0041]], device='mps:0')

In [72]:
outputs.logits_per_image.softmax(dim=1)

tensor([[0.9949, 0.0051]], device='mps:0')

In [ ]:
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

print(probs)

In [ ]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = "mps" if torch.backends.mps.is_available() else DEVICE

# attention_type = "flash_attention_2" if DEVICE == "cuda" else "eager"
# processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")
# model = AutoModelForVision2Seq.from_pretrained("HuggingFaceTB/SmolVLM-Instruct",
#                                                torch_dtype=torch.bfloat16,
#                                                _attn_implementation=attention_type).to(DEVICE)

# model_path = 'HuggingFaceTB/SmolVLM-Instruct'
# quant_path = 'qwen-instruct-v0.2-awq'
# quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }
#
# # Загружаем модель
# model = AutoAWQForCausalLM.from_pretrained(
#     model_path, **{"low_cpu_mem_usage": True, "use_cache": False}
# )
# tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
#
# # Квантизация
# model.quantize(tokenizer, quant_config=quant_config)
#
# # Сохраняем квантованную модель
# model.save_quantized(quant_path)
# tokenizer.save_pretrained(quant_path)

# Paper Processor pipeline

In [1]:
import sys
import os

sys.path.append('..')

In [2]:
from src.services.file_processor.client import PaperProcessor

/Users/nikitayatchenko/PycharmProjects/paper_bot


In [4]:
pp = PaperProcessor()

2025-04-21 13:07:53,801 - paper_bot - INFO - Loading model
2025-04-21 13:07:55,094 - paper_bot - INFO - Loading processor


In [6]:
pp.process()

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4030 > 77). Running this sequence through the model will result in indexing errors


ValueError: Sequence length must be less than max_position_embeddings (got `sequence length`: 4311 and max_position_embeddings: 77